# 第一題[myknn_regressor]

＊在這裡說明我要幹嘛＊

In [20]:
# given code
import pickle
from sklearn import preprocessing
import numpy as np

# Load data
with open('msd_data1.pickle', 'rb') as fh1:
    msd_data = pickle.load(fh1)

doscaling = 1

if (doscaling == 1):
    xscaler = preprocessing.StandardScaler().fit(msd_data['X_train'])
    #standardize feature values
    X_train = xscaler.transform(msd_data['X_train'])
    X_test = xscaler.transform(msd_data['X_test'])
else:
    X_train = msd_data['X_train']
    X_test = msd_data['X_test']

Y_train = msd_data['Y_train']
Y_test = msd_data['Y_test']

# observation
print(Y_train)

[1996. 2001. 2003. ... 2009. 1982. 1981.]


In [107]:
class myknn_regressor():
    def __init__(self, n_neighbors, mean_type):
        """mean_type can be equal_weight or remove_outliers.
                              equal_weight use the same weight for all neighbors.
                              remove_outliers remove neighbors out in [Q1 - 1.5 IQR, Q3 + 1.5IQR]."""
        ### Add your code here ###
        self.n_neighbors = n_neighbors
        self.mean_type = mean_type

    def fit(self, x_train, y_train):
         ### Add your code here ###
            ### 整理成numpy array ###
            self.x_train = x_train
            self.y_train = y_train

    def predict(self, x_test):
        """use remove_outliers only if k>=10"""
        ### Add your code here ###
        # 計算 x_test 跟 x_train 每個點的距離
        self.x_test = x_test
        k = self.n_neighbors
        x_train = self.x_train
        y_train = self.y_train
        mean_type = self.mean_type
        max_dis = 5.0 #存k個中最遠的
        y_predict = np.empty((len(x_test),1), dtype=int)
        
        for a in range(20):
            ### 現在都只有 x_test[0] ###
            neighbors = np.empty((k,2),dtype=float )
            # 寫一個只做 x_test[0], 把 x_train 掃一輪找到真的KNN （剛好先填滿neighbors

            test_counter = 0
            dis_temp = 0.0
            for j in x_train:
                if(test_counter<k): #前k個 x_train
                    for l in range(len(j)):
                        dis_temp = dis_temp + (((x_test[a][l]-j[l])**2)**0.5) #計算兩個點之間的距離
                    dis_temp = dis_temp/len(j) #平均
                    if (dis_temp<max_dis): #更新極值
                        max_dis = dis_temp
                    neighbors[test_counter][0]=dis_temp
                    neighbors[test_counter][1]=y_train[test_counter]

                else:
                    for l in range(len(j)):
                        dis_temp = dis_temp + (((x_test[a][l]-j[l])**2)**0.5) #計算兩個點之間的距離
                    dis_temp = dis_temp/len(j) #平均
                    # 開始跟前k個比較，有沒有比前k個中最遠的更近，有的話就更新
                    if (dis_temp<max_dis): #更新極值 note:有必要這個步驟嗎？
                        max_dis = dis_temp
                    for i in range(k):
                        if(dis_temp < neighbors[i][0]):
                            neighbors[i][0]=dis_temp
                            neighbors[i][1]=y_train[test_counter]
                            break
                test_counter+=1
                dis_temp = 0.0

            # 判斷 mean_type
            if(mean_type == "remove_outliers"):

                #新增一個 array 排序，找出 quantile
                quantile = np.empty((k,1),dtype=float)
                ro_outcome = 0
                for i in range(k):
                    ro_outcome += neighbors[i][1]
                    quantile[i] = neighbors[i][1]

                for m in range(k):
                    for n in range(k-1):
                        if(quantile[n]>quantile[n+1]):
                            temp = quantile[n]
                            quantile[n] = quantile[n+1]
                            quantile[n+1] = temp

                #print(quantile)
                Q1 = int((k+1)*0.25)
                Q3 = int((k+1)*0.75)
                IQR = quantile[Q3]-quantile[Q1]
                Out_low = quantile[Q1]-(IQR*1.5)
                Out_high = quantile[Q3]+(IQR*1.5)
                #print(IQR,Out_low, Out_high)

                if(k<10):
                    ro_outcome = ro_outcome/k
                else:
                    ro_outcome = 0
                    ro_count = 0
                    for i in range(k):
                        if(neighbors[i][1]>=Out_low) and (neighbors[i][1]<=Out_high):
                            ro_outcome += neighbors[i][1]
                            ro_count += 1
                    ro_outcome = ro_outcome/ro_count
                y_predict[a]=ro_outcome



            else:
                ew_outcome = 0
                for i in range(k):
                    ew_outcome += neighbors[i][1]
                ew_outcome = ew_outcome/k
                y_predict[a]=ew_outcome
        
        ### 加入全部的 x_test 之後，要計算 RMSE 然後列出前20個預測結果
        print(y_predict[:20])

In [108]:
### test arc ###
myknn = myknn_regressor(20, "remove_outliers")
myknn.fit(X_train, Y_train)
ypred = myknn.predict(X_test)

[[1962]
 [1975]
 [1977]
 [1989]
 [1971]
 [1970]
 [1967]
 [1996]
 [2001]
 [2003]
 [2000]
 [1992]
 [1971]
 [1992]
 [1978]
 [1978]
 [1962]
 [2000]
 [2002]
 [1995]]
